<p>There are different tensorflow API's. The lowest level API is Tensorflow Core. Some higher level API's are tf.estmator</p>

### Tensors
<p>The rank of a tensor is its number of dimensions.</p>

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np

<p>Tensorflow core programs can be thought of as first building the computation graph and then runing the computation graph</p>

In [3]:
node1 = tf.constant(3. , dtype = tf.float32)
node2 = tf.constant(4.)
print(node1 , node2)

Tensor("Const_1:0", shape=(), dtype=float32) Tensor("Const_2:0", shape=(), dtype=float32)


In [5]:
# To run the computation graph we use Session
sess = tf.Session()
print(sess.run([node1 , node2]))

[3.0, 4.0]


In [7]:
node3 = tf.add(node1 , node2)
print("node3:" , node3)
print("sess.run(node3):" , sess.run(node3))

node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 7.0


<p>To view the computation graph we use tensorboard</p>
<p>Now to create variables without assigning them values we use placehodlers</p>

In [8]:
a = tf.placeholder(dtype = tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b

In [9]:
print(sess.run(adder_node , {a:3. , b:4.}))
print(sess.run(adder_node , {a:[1. , 3.] , b:[4. , 7.]}))

7.0
[  5.  10.]


In [10]:
# To add trainable parameters in tensorflow we have to use Variables
W = tf.Variable([.3] , dtype=tf.float32)
b = tf.Variable([-.3] , dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

In [13]:
# Constants are initialized using tf.constant and you cannot reinitialize them
# To initialize variables we use tf.global_varibales_initializer() 
# as they are not initialized when you call tf.Variable
init = tf.global_variables_initializer()
sess.run(init)

In [14]:
print(sess.run(linear_model , {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [15]:
# Now to update our model we have to make a loss function
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss , {x:[1,2,3,4] , y:[0,-1,-2,-3]}))

23.66


In [18]:
# When you want to assign a value to a variable
fixW = tf.assign(W , [-1.])
fixb = tf.assign(b , [1.])
sess.run([fixW , fixb])
print(sess.run(loss , {x:[1,2,3,4] , y:[0,-1,-2,-3]}))

0.0


#### tf.train API

In [19]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [20]:
sess.run(init)
for _ in range(1000):
    sess.run(train , {x:[1,2,3,4] , y:[0,-1,-2,-3]})
    
print(sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


#### tf.estimator

In [ ]:
# Simplifies the mechanics of machine learning i.e running training,evaluation loops and managing data sets
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

#tf.estimate contains many simple machine learning models like linear regression, linear classification
estimator = tf.estimator.LinearRegressor(feature_columns = feature_columns)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.estimator.inputs.numpy_input_fn({"x":x_train}, y_train, batch_size=4, num_epochs=None , shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x":x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=True)
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x":x_eval}, y_eval, batch_size=4, num_epochs=None, shuffle=True)

estimator.train(input_fn=input_fn, steps=1000)

train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)

print("train metrics: %r"%train_metrics)
print("eval metrics: %r"%eval_,metrics)

In [ ]:
# To create custom models use tf.estiamtor.Estimator
def model_fn(features, labels, mode):
    W = tf.Variable("W" , [1] , dtype=tf.float32)
    b = tf.Variable("b" , [1] , dtype=tf.float32)
    t = w*featues['x'] + b
    
    loss = tf.reduce_sum(tf.square(y-labels))
    
    #training global sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss) , tf.assign_add(global_step , 1))
    
    #EstimateSpec connects subgraphs we built to the appropriate funcitonality
    return tf.estimator.EstimateSpec(
    mode=mode,
    prediction=y,
    loss=loss,
    train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

estimator.train(input_fn=input_fn, steps=1000)
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

## MNIST For ML Beginners
<p>It is the standard dataset that is used to quickly train your models (computer vision dataset)</p>
<p>MNIST is analogous to 
<python>print("Hello, World")</python>
</p>

In [2]:
# To implement Softmax Regression for hand written digit recognition

# Get the data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/" , one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<ul>About the dataset
<li>Contains 55000 training , 5000 validation and 10000 test examples</li>
<li>mnist.train.images and mnist.train.labels to get your images and labels</li>
<li>Images of size 28x28</li>
<li>'One-hot vector means a vector 0 in most dimensions and 1 in a single dimension like [0,0,0,0,1,0,0]</li>
</ul>

The key to the speed of tensorflow is use of efficient numerical computing with libraries like NumPy. Now to do these computations different from Python it will have to switch back and forth between the two and it will lead to a lot of overhead. To avoid this, instead of using a single expensive operation independently from Python, Tensorflow uses a graph of operations that run entirely outside Python.

In [26]:
x = tf.placeholder(dtype=tf.float32 , shape=[None , 784])
W = tf.Variable(tf.zeros([784 , 10]))
b = tf.Variable(tf.zeros([10]))# Implementation of cross_entropy
y_hat = tf.placeholder(tf.float32 , [None , 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_hat * tf.log(y), reduction_indices=[1]))
# reduction_indices can be thought of ax axis of a tensor
y = tf.nn.softmax(tf.matmul(x , W)+b)

In [27]:
# Implementation of cross_entropy
y_hat = tf.placeholder(tf.float32 , [None , 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_hat * tf.log(y), reduction_indices=[1]))
# reduction_indices can be thought of ax axis of a tensor

In [28]:
train = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
# 0.5 is the learning rate

In [29]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for _ in range (10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train, feed_dict={x: batch_xs, y_hat: batch_ys})

In [32]:
correct_prediction = tf.equal(tf.argmax(y,1) , tf.argmax(y_hat,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
print(sess.run(accuracy , feed_dict={x:mnist.test.images , y_hat:mnist.test.labels}))

0.9263


# Deep MNIST for Experts

In [34]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data' , one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [37]:
x = tf.placeholder(tf.float32 , shape=[None,784])
y_hat = tf.placeholder(tf.float32 , shape=[None , 10])
W = tf.Variable(tf.zeros([784 , 10]))
b = tf.Variable(tf.zeros([10]))

In [39]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [42]:
y = tf.matmul(x,W)+b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_hat , logits = y))

In [43]:
train = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for _ in range(10000):
    batch = mnist.train.next_batch(100)
    sess.run(train , feed_dict={x:batch[0] , y_hat:batch[1]})

In [44]:
correct_prediction = tf.equal(tf.argmax(y,1) , tf.argmax(y_hat , 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_hat:mnist.test.labels}))

0.926


In [55]:
# Now the convolutional neural network approach
def weight_variable(shape):
    initial = tf.truncated_normal(shape , stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.truncated_normal(shape , stddev=0.1)
    return tf.Variable(initial)

In [69]:
def conv2d(x , W):
    return tf.nn.conv2d(x , W , strides = [1,1,1,1] , padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x , strides = [1,2,2,1] , ksize = [1,2,2,1] , padding='SAME')

In [83]:
x_conv = tf.placeholder(tf.float32 , shape=[None , 784])
y_hat_conv = tf.placeholder(tf.float32 , shape=[None , 10])

W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x_conv , [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image , W_conv1) + b_conv1)
h_pool2 = max_pool_2x2(h_conv1)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1 , keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
cross_entropy_conv = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_hat_conv, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy_conv)
correct_prediction_conv = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_hat_conv, 1))
accuracy_conv = tf.reduce_mean(tf.cast(correct_prediction_conv, tf.float32))


sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for a in range(20000):
    batch = mnist.train.next_batch(50)
    if (a % 100 == 0):
        train_accuracy = accuracy.eval(feed_dict={x_conv: batch[0], y_hat_conv: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x_conv: batch[0], y_hat_conv: batch[1], keep_prob: 0.5})

print('test accuracy %g' % accuracy.eval(feed_dict={x_conv: mnist.test.images, y_hat_conv: mnist.test.labels, keep_prob: 1.0}))

# Estimators

### Pre made Estimators
<p>Don't have to worry about graph or sessions since Estimators handle all the "plumbing" for you</p>
<p>Structure of a pre-made estimator<br>
<ol>
<li>Write one or more dataset importing functions where it should return two things, a dictionary with keys are feature names and values are the Tensrs and a Tensor containing one or more labels</li>
<li>Define the feature columns. tf.feature_column identifies a feature name and its type and any input preprocessing.</li>
<li>Instantiate the relevant pre-made Estimator.Instantialization means creating a object</li>
<li>Call a training, evaluation or inference method</li>
</ol></p>

In [ ]:
# 1 step
def input_fn(dataset):
    # manipulate it to extract feature names and the labels
    return feature_dict , label

# 2 step
a = tf.feature_column.numeric_column('a')
b = tf.feature_column.numeric_column('b')

# 3 step
estimator = tf.estimator.Estimate.LinearClassifier(feature_columns=[a,,b])

# 4 step
estimator.train(input_fn = my_training_set , steps =200)

### Custom Estimators
<p>Estimators use model-function to work, when working with Pre-made Estimators someone has already implemented them and in Custom Estimators you have to implement them</p>

# Tensors

Tensorflow is a framework to define and run computations involving tensors. A tensor is a generalization of vectors and matrices to potentially higher dimensions. tf.Tensor defines a computation that will eventuallly produce a value. Tensorflow programs work by first creating a computation graph for tf.Tensor and then by running parts of this graph to achieve the desired result.</p>
<p>tf.Tensor has a data_type and a shape</p>
<p>tf.Variable , tf.Placeholder , tf.Constant , tf.SparseTensor</p>
<h3>Rank</h3>
<ul>
<li>0 - scalar</li>
<li>1 - vector</li>
<li>2 - matrix </li>
<li>3 - 3-Tensor</li>
</ul>

Tensor("Rank_1:0", shape=(), dtype=int32)


In [13]:
# To evaluate a tensor use Tensor.eval. It works only when a default tf.Session is active

In [14]:
# To print a tensor during debugging
t = tf.constant([1,2,3] , dtype=tf.float32)
print (t)

Tensor("Const_8:0", shape=(3,), dtype=float32)


In [27]:
# Now to correctly print the result
t = tf.constant([1,2,3] , dtype=tf.float32)
t = tf.Print(t , [t])
result = t+1

In [17]:
print(result)

Tensor("add:0", shape=(3,), dtype=float32)


# Variables

In [26]:
# To create a variable use tf.get_variable
variable = tf.get_variable('variable' , [1,2,3])
# It will create a variable whose shape is [1,2,3]

<p>As tensorflow programs may create variables on their own for graph compuation, it is sometimes usefule to to access them from a single place i.e. why we use collections, which are namedlist of tensors, such as tf.Variable instances</p>
<p>By default ever Variable is placed in two collections tf.GraphKeys.GLOBAL_VARIBALES(that can be shared among multiple devices) and tf.GraphKeys.TRAINABLE_VARIABLES(variables for which tensorflow will calculate gradients). If you don't want your variables to be trainable than add them to tf.GraphKeys.LOCAL_VARIABLES or give a argument trainable=False</p>

In [ ]:
my_local = tf.get_variable('my_local' , shape=() , collections=[tf.GraphKeys.LOCAL_VARIABLES])

In [33]:
my_non_trainable = tf.get_variable('my_non_trainable' , shape=() , trainable=False)

In [ ]:
# you can also explicitly add varibales to a collection
tf.add_to_collection('collection_name' , my_variable)

#To retrieve a list of all the variables in a collection
tf.get_collection('my_collection')

In [ ]:
# To select the device you want your varibale to run
with tf.device('/device:GPU:1'):
    v = tf.get_variable('v' , [1])

### Initializing Variables
to initialize all trainable variables in one go use tf.global_variables_initializer()<br>
If you want to initialize them use sess.run(my_variable.initializer)<br>
To print the variables not initialized print(sess.run(tf.report_uninitialized_variables()))<br>
By default tf.global_variables_initalizer does not specify the order in which variables are intitialized, so if the value of a variable depends on another variable than you can use initializer = variable_name.initialized_value()

In [34]:
#  To assign a value to a variable use tf.assign , tf.assign_add

In [35]:
# Now to reuse variables you have to just change the scope

test_variable = tf.get_variable('test_variable' , shape=())

In [ ]:
# Now calling it again will result in an erro
test_variable = tf.get_variable('test_variable' , shape=())

In [43]:
# You can just lower level API for variable (explicitly)
test = tf.Variable('test' , [1,2,3])
test = tf.Variable('test' , [1,2,3])

In [ ]:
# You can wrap them in tf.variable_scope objects (implicitly)
with tf.variable_scope('model' , reuse=True):
    

# Graphs and Sessions

<p>You first make the dataflow graph and then make session to run parts of it. In higher levels API's it is already carried bythe API.</p>
<p>The edges of the graph represent the data and the nodes represent the operation to be done on the data. Graph is the prefereed way in tensorflow because it enables parallel computing, distributed execution, compilation and portability</p>

### tf.Graph
Conatins
<ul>
<li>Graph structure</li>
<li>Graph Collections</li>
</ul>

tf.Operation is used to create a node<br>
tf.Tensor is used to create a edge<br>
E.g when you call tf.constant(4.0) It creates a tf.Operation and a edge whose value is 4 and adds it to the 'default graph'

When you do any tf.Operation you should give it a argument 'name' as it will enable you to look in the default graph easily.

In [50]:
c_0 = tf.constant(0 , name='c')
print (c_0)

c_1 = tf.constant(2 , name='c' , dtype=tf.float64)
print(c_1)

Tensor("c_4:0", shape=(), dtype=int32)
Tensor("c_5:0", shape=(), dtype=float64)


In [ ]:
def check_number (x) :
    count4 = 0
    count7 = 0
    numCount = 0
    while (x != 0):
        temp = x%10
        if (temp == 4):
            count4 = count4 + 1
        if (temp == 7):
            count7 = count7 + 1
        numCount = numCount + 1
    if (count4 == count7 and count4 + count7 == numCount):
        return 1
    return 0

n = int(input())
laptop = {}
for _ in range(n):
    a = input().split(' ')
    Name = a[0]
    Price = int(a[1])
    laptop[Name] = Price

check = []
for key in laptop:
    check.append
    